In [ ]:
from bs4 import BeautifulSoup  ## the BeautifulSoup library for scraping from the bs4 package
import requests ## Establish website connection using the requests library
import pandas as pd
import numpy as np
import re ## RegEx for pattern matching

import requests
from itertools import cycle
import time
import random

import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

key = 'Input own Key'

Mounted at /content/gdrive


# Section 1
- Crawl through the WhoSampled.com site to find songs with the Amen break

In [ ]:
site = 'https://www.whosampled.com/The-Winstons/Amen,-Brother/sampled/?cp=5'

payload = { 'api_key': key, 'url': site, 'country_code': 'eu' }
r = requests.get('https://api.scraperapi.com/', params=payload)
print(r)

In [ ]:
soup = BeautifulSoup(r.text)
lst = soup.find_all('a',class_ ="trackName playIcon")

links = []

for i in lst:
  links.append(i.get('href'))

In [ ]:
site = 'https://www.whosampled.com/The-Winstons/Amen,-Brother/sampled/'
Amen_links = []
success_list = []

for i in range(5,100):

  rand = random.randint(3, 15)
  time.sleep(rand)

  add = '?cp=' + str(i)
  link = site + add

  payload = { 'api_key': 'a3298e85be2b1bde1a8b99bbcc7ba181', 'url': link, 'country_code': 'eu' }
  r = requests.get('https://api.scraperapi.com/', params=payload)

  if r.status_code != 200:
    print('code is', r.status_code)
    print('iteration number is', i)
  else:
    print('success!', i)
    success_list.append(i)

  soup = BeautifulSoup(r.text)
  lst = soup.find_all('a',class_ ="trackName playIcon")

  for i in lst:
    Amen_links.append(i.get('href'))
    pass



In [ ]:
df = pd.DataFrame(columns = ['link'])
df['link'] = Amen_links
df['link'] = df['link'].apply(lambda x: 'https://www.whosampled.com' + x)
df.to_csv('Amen_links', index = False)

In [ ]:
success_list
np.savetxt("Successful_Amens.txt", success_list, delimiter=",", fmt='%s')

# Section 2
- Scrape the youtube link to songs with the Amen break, along with supplementary information.

In [ ]:
df = pd.read_csv('Amen_links')
df_songs = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt')

In [ ]:
def get_info(soup):

  artist_link = soup.find('a',itemprop ="item").get('href')
  vid_number = len(soup.find_all('div', class_ ="embed-placeholder youtube-placeholder"))

  if vid_number == 2:
    ytube_link = soup.find('div', class_ ="embed-placeholder youtube-placeholder").get('data-id')
    time_stamp = soup.find('span',class_ = "timing dest-timing").text
    time_stamp2 = soup.find('span',class_ = "timing source-timing").text

    return [artist_link,ytube_link,time_stamp,time_stamp2, True]
  return [artist_link,'','','', False]

In [ ]:

for i in range(1248):

  rand = random.randint(3, 8)
  time.sleep(rand)

  link = df['link'][i]

  payload = { 'api_key': 'a3298e85be2b1bde1a8b99bbcc7ba181', 'url': link, 'country_code': 'eu' }
  r = requests.get('https://api.scraperapi.com/', params=payload)

  if r.status_code != 200:
    print('code is', r.status_code, 'iteration', i)
    df_songs.loc[i] = [link,'','','','', False]


  else:
    print('success!', i)

    soup = BeautifulSoup(r.text)
    info = get_info(soup)
    info = [link] + info
    df_songs.loc[i] = info


  if i%50 == 0:
     df_songs.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)

df_songs.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)




# Section 3


In [ ]:
downloaded_info = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/Amen_download_info.txt')
downloaded_info

In [ ]:
url = 'https://www.whosampled.com/song-tag/Drum%20Break/2/'
apikey = key
params = {
    'url': url,
    'apikey': apikey,
}
response = requests.get('https://api.zenrows.com/v1/', params=params)
print(response)

<Response [200]>


In [ ]:
soup = BeautifulSoup(response.text)

artists = soup.find_all('span', class_ ="trackArtistName")
for i,j in enumerate(artists):
  artists[i] = [j.find('a').get('href'),0]

info = artists

In [ ]:
songs = soup.find_all('a', itemprop = 'url')
for i,j in enumerate(songs):
  info[i][1] = j.get('href')

info

In [ ]:
def get_negative_info(soup):

  artist = soup.find('span', class_ ="trackArtistName").find('a').get('href')
  song = soup.find('a', itemprop = 'url').get('href')

  return [song, artist]


In [ ]:
negative_df = pd.DataFrame(columns = ['Original_Artist','Link'])

In [ ]:

for k in range(75):

  rand = random.randint(1, 5)
  time.sleep(rand)

  url = 'https://www.whosampled.com/song-tag/Drum%20Break/' + str(k) + '/'
  apikey = key
  params = {
      'url': url,
      'apikey': apikey,
  }
  response = requests.get('https://api.zenrows.com/v1/', params=params)
  soup = BeautifulSoup(response.text)


  artists = soup.find_all('span', class_ ="trackArtistName")
  for i,j in enumerate(artists):
    artists[i] = j.find('a').get('href')

  songs = soup.find_all('a', itemprop = 'url')
  for i,j in enumerate(songs):
    negative_df.loc[len(negative_df)] = [artists[i], j.get('href')]



In [ ]:
negative_df

,Original_Artist,Link
0,/Sugarhill-Gang/,/Sugarhill-Gang/Rapper%27s-Delight/
1,/Barry-White/,/Barry-White/I%27m-Gonna-Love-You-Just-a-Littl...
2,/Eddie-Johns/,/Eddie-Johns/More-Spell-on-You/
3,/Skull-Snaps/,/Skull-Snaps/It%27s-a-New-Day/
4,/Sly-%26-the-Family-Stone/,/Sly-%26-the-Family-Stone/Sing-a-Simple-Song/
...,...,...
705,/Eddie-Simpson/,/Eddie-Simpson/Big-Black-Funky-Slave/
706,/Ely-Barra/,/Ely-Barra/As-Turbinas-Est%C3%A3o-Ligadas/
707,"/Ashton,-Gardner-%26-Dyke/","/Ashton,-Gardner-%26-Dyke/Let-It-Roll/"
708,/Leroy-Hutson/,/Leroy-Hutson/All-Because-of-You/


In [ ]:
negative_df.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info_new.txt', index = False)

In [ ]:
negative_df['Link'] = negative_df['Link'].apply(lambda x: 'https://www.whosampled.com' + x + 'sampled/')
negative_df = negative_df[['Original_Artist','Link']]
negative_df

# Section 4


In [ ]:
downloaded_info = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/Non_Amen_links.txt')
downloaded_info['Link'] = downloaded_info['Link'].apply(lambda x: 'https://www.whosampled.com' + x)
downloaded_info = downloaded_info[['Original_Artist','Link']]
downloaded_info

In [ ]:
def extract_3_related(soup):

  songz = []
  expected_vals = 0

  for spoon in soup.find_all('section', class_="section")[0]:

    try:
      txt = spoon.find('h3', class_ = "section-header-title").text
      if txt.split(' ')[0] == 'Sampled':

        expected_vals = int(txt.split(' ')[2])
        expected_vals = min(expected_vals, 3)

        for i in spoon.find_all('a'):

          line = i.get('href')
          if line.split('/')[1] == 'sample':
            songz.append(line)


        info = []
        for i in spoon.find_all('div', class_="trackBadge"):

          sample_type = i.find('span', class_="topItem").text
          genre = i.find('span', class_="bottomItem").text

          info.append([sample_type, genre])

    except:
      pass

  pages = (songz[::2])
  if expected_vals == 0:
    return [], ['none', 0, 0], []

  if len(pages) == expected_vals:
    link_number = 'expected'
  elif len(pages) > expected_vals:
    link_number = 'too many'
  elif len(pages) < expected_vals:
    link_number = 'too little'

  return pages, [link_number, len(pages), expected_vals], info

#lst, vals, song_info = extract_3_related(soup2)
#print(lst,vals, song_info)

In [ ]:
sample_page_links = pd.DataFrame(columns = ['Original_artist','Link','Page','Genre','Sample_type','outcome','expected_vals','actual_vals'])
sample_page_links.loc[len(sample_page_links)] = list(downloaded_info.loc[0]) + [lst[0]] + [song_info[0][0]] + [song_info[0][1]] + vals
sample_page_links

,Original_artist,Link,Page,Genre,Sample_type,outcome,expected_vals,actual_vals
0,/Sugarhill-Gang/,https://www.whosampled.com/Sugarhill-Gang/Rapp...,/sample/24272/Yellowman-Beat-It-Michael-Jackso...,Vocals / Lyrics,Reggae / Dub,expected,3,3


In [ ]:
list(downloaded_info.loc[0]) + [lst[0]] + [song_info[0][0]] + [song_info[0][1]] + vals

In [ ]:
sample_page_links = pd.DataFrame(columns = ['Original_artist','Link','Page','Genre','Sample_type','outcome','expected_vals','actual_vals'])

In [ ]:
offset = 507

for ii,url in enumerate(downloaded_info['Link'][offset:]):

  i = offset + ii
  rand = random.randint(1, 5)
  time.sleep(rand)

  apikey = 'b86fb84806eec8899e7b531d9644a18e18eb2b95'
  params = {
      'url': url,
      'apikey': apikey,
  }
  r = requests.get('https://api.zenrows.com/v1/', params=params)
  soup = BeautifulSoup(r.text)

  if r.status_code == 200:

    lst, vals, song_info = extract_3_related(soup)

    print(i,'Success',vals)

    for j in range(len(lst)):
      sample_page_links.loc[len(sample_page_links)] = list(downloaded_info.loc[i]) + [lst[j]] + [song_info[j][0]] + [song_info[j][1]] + vals

  else:

    print(i,'Failed',r.status_code)

  if i%50 == 0:
    sample_page_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info_new.txt', index = False)



In [ ]:
sample_page_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info_new.txt', index = False)

# Section 5


In [ ]:
Negative_pages = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/Non_Amen_page_links.txt')
#downloaded_info['Link'] = downloaded_info['Link'].apply(lambda x: 'https://www.whosampled.com' + x)
#downloaded_info = downloaded_info[['Original_Artist','Link']]
Negative_pages = Negative_pages.loc[3:]
Negative_pages['Page'] = Negative_pages['Page'].apply(lambda x : 'https://www.whosampled.com' + x)
Negative_pages = Negative_pages.reset_index().iloc[:,1:]
Negative_pages

In [ ]:
def provisional(soup,n,c):

  ytube_link = soup.find_all('div', class_ ="embed-placeholder youtube-placeholder")[n].get('data-id')
  time_stamp = soup.find('span',class_ = c).text

  return [ytube_link, time_stamp]

In [ ]:
def get_negative_page_info(soup):

  Amen_locator = False
  for song in soup.find_all('a',class_ = "trackName playIcon"):
    title = song.get('title')
    if title == "The Winstons's Amen, Brother":
      Amen_locator = True

  vid_number = len(soup.find_all('div', class_ ="embed-placeholder youtube-placeholder"))
  artist_link = soup.find('a',itemprop ="item").get('href')

  time_stamp = soup.find('span',class_ = "timing dest-timing")
  time_stamp2 = soup.find('span',class_ = "timing source-timing")

  if time_stamp and time_stamp2:
    if vid_number != 2:
      output = [artist_link, '','','','','inconsistent']
    else:
      a = provisional(soup,0,'timing dest-timing')
      b = provisional(soup,1,'timing source-timing')
      output = [artist_link] + a+b + ['success']

  elif time_stamp:
    if vid_number != 1:
      output = [artist_link, '','','','','inconsistent']
    else:
      a = provisional(soup,0,'timing dest-timing')
      output = [artist_link] + a + ['',''] + ['partial-1']

  elif time_stamp2:
    if vid_number != 1:
      output = [artist_link, '','','','','inconsistent']
    else:
      b = provisional(soup,0,'timing source-timing')
      output = [artist_link] + ['',''] + b + ['partial_2']

  else:
    output = [artist_link, '','','','','None']

  output += [Amen_locator]

  return output


In [ ]:
# plan - find if timing dest or source occurs or both
#        use to assign links

In [ ]:
#Negative_ytube_links = pd.DataFrame(columns = (list(Negative_pages.columns) + ['New_artist','ytube_link_new','timestamp_new','ytube_link_old', 'timestamp_old','success','Amen_relation']))

In [ ]:
Negative_ytube_links = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/Non_Amen_samples.txt')
Negative_ytube_links

In [ ]:
for i in range(1302,len(Negative_pages)):
#for i in failed_inds:

  rand = random.randint(1, 5)
  time.sleep(rand)

  link = Negative_pages['Page'][i]

  apikey = key
  params = {
      'url': link,
      'apikey': apikey,
  }
  r = requests.get('https://api.zenrows.com/v1/', params=params)
  soup = BeautifulSoup(r.text)

  if r.status_code == 200:
    output = get_negative_page_info(soup)
    print(i,output[-2],output[-1])

  else:
    print(i,'failed',r.status_code)
    output = ['']*7

  Negative_ytube_links.loc[i] = list(Negative_pages.loc[i]) + output

  if i%50 == 0:
     Negative_ytube_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)

Negative_ytube_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)




In [ ]:
Negative_ytube_links['success'].value_counts()

success      1900
partial_2     101
partial-1      27
None            3
Name: success, dtype: int64

In [ ]:
Negative_pages['Sample_type'].value_counts()

Hip-Hop / Rap / R&B     1443
Electronic / Dance       390
Rock / Pop               108
Soul / Funk / Disco       31
Other                     28
Soundtrack / Library      12
Reggae / Dub               7
Jazz / Blues               6
World / Latin              2
Easy Listening             2
Spoken Word                1
Gospel                     1
Name: Sample_type, dtype: int64

In [ ]:
apikey = key
params = {
    'url': 'https://www.whosampled.com/The-Winstons/Amen,-Brother/',
    'apikey': apikey,
}
r = requests.get('https://api.zenrows.com/v1/', params=params)
print(r)

<Response [200]>


In [ ]:
df = pd.read_csv('failed_downloads.txt')
df.loc[0][0].split(' ')[1]

failed_inds = []
for i in range(len(df)):
  if df.loc[i][0].split(' ')[1] == 'failed':
    failed_inds.append(int(df.loc[i][0].split(' ')[0]))

failed_inds


In [ ]:
df['200 success False'].value_counts()

201 success False    1
933 success False    1
939 success False    1
938 failed 422       1
937 success False    1
                    ..
574 success False    1
575 success False    1
576 success False    1
577 success False    1
1301 failed 402      1
Name: 200 success False, Length: 1101, dtype: int64

# Section 6


In [ ]:
link = 'https://www.whosampled.com/song-tag/Single/8/'

payload = { 'api_key': key, 'url': link, 'country_code': 'eu' }
r = requests.get('https://api.scraperapi.com/', params=payload)
r

<Response [200]>

In [ ]:
general_df = pd.DataFrame(columns = ['Original_Artist','Link'])

In [ ]:
from ast import keyword
for k in range(2,125):

  rand = random.randint(1, 3)
  time.sleep(rand)

  link = 'https://www.whosampled.com/song-tag/Single/' + str(k)

  payload = { 'api_key': key, 'url': link, 'country_code': 'eu' }
  r = requests.get('https://api.scraperapi.com/', params=payload)

  if r.status_code == 200:
    soup = BeautifulSoup(r.text)

    artists = soup.find_all('span', class_ ="trackArtistName")
    for i,j in enumerate(artists):
      artists[i] = j.find('a').get('href')

    songs = soup.find_all('a', itemprop = 'url')
    for i,j in enumerate(songs):
      general_df.loc[len(general_df)] = [artists[i], j.get('href')]

  print(k, r.status_code)

In [ ]:
general_df.to_csv('/content/gdrive/MyDrive/Amen_Remastered/General_links.txt', index = False)

# Section 7


In [ ]:
downloaded_info = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/General_links.txt')
downloaded_info['Link'] = downloaded_info['Link'].apply(lambda x: 'https://www.whosampled.com' + x)
downloaded_info = downloaded_info[['Original_Artist','Link']]
downloaded_info

,Original_Artist,Link
0,/Drake/,https://www.whosampled.com/Drake/Pound-CakePar...
1,/Dr.-Dre/,https://www.whosampled.com/Dr.-Dre/The-Next-Ep...
2,/James-Brown/,https://www.whosampled.com/James-Brown/Get-Up-...
3,/Bad-Bunny/,https://www.whosampled.com/Bad-Bunny/MONACO/
4,/Coi-Leray/,https://www.whosampled.com/Coi-Leray/Players/
...,...,...
1225,/Destiny%27s-Child/,https://www.whosampled.com/Destiny%27s-Child/E...
1226,/Missy-Elliott/,https://www.whosampled.com/Missy-Elliott/One-M...
1227,/Smile.dk/,https://www.whosampled.com/Smile.dk/Butterfly/
1228,/Timbaland/,https://www.whosampled.com/Timbaland/The-Way-I...


In [ ]:
sample_page_links = pd.DataFrame(columns = ['Original_artist','Link','Page','Genre','Sample_type','outcome','expected_vals','actual_vals'])

In [ ]:
offset = 86

for ii,url in enumerate(downloaded_info['Link'][offset:]):

  i = offset + ii
  rand = random.randint(1, 3)
  time.sleep(rand)

  payload = { 'api_key': key, 'url': url, 'country_code': 'eu' }
  r = requests.get('https://api.scraperapi.com/', params=payload)
  soup = BeautifulSoup(r.text)

  if r.status_code == 200:

    lst, vals, song_info = extract_3_related(soup)

    print(i,'Success',vals)

    for j in range(len(lst)):
      sample_page_links.loc[len(sample_page_links)] = list(downloaded_info.loc[i]) + [lst[j]] + [song_info[j][0]] + [song_info[j][1]] + vals

  else:

    print(i,'Failed',r.status_code)

  if i%50 == 0:
    sample_page_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info_new.txt', index = False)

In [ ]:
sample_page_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info_new.txt', index = False)

# Section 8


In [ ]:
General_pages = pd.read_csv('/content/gdrive/MyDrive/Amen_Remastered/General_page_links.txt')
General_pages['Page'] = General_pages['Page'].apply(lambda x : 'https://www.whosampled.com' + x)
#Negative_pages = Negative_pages.reset_index().iloc[:,1:]
General_pages['Page'][0]


In [ ]:
General_ytube_links = pd.DataFrame(columns = (list(Negative_pages.columns) + ['New_artist','ytube_link_new','timestamp_new','ytube_link_old', 'timestamp_old','success','Amen_relation']))

In [ ]:
for i in range(1276,len(General_pages)):
#for i in failed_inds:

  rand = random.randint(1, 3)
  time.sleep(rand)

  link = General_pages['Page'][i]

  payload = { 'api_key': key, 'url': link, 'country_code': 'eu' }
  r = requests.get('https://api.scraperapi.com/', params=payload)
  soup = BeautifulSoup(r.text)

  if r.status_code == 200:
    output = get_negative_page_info(soup)
    print(i,output[-2],output[-1])

  else:
    print(i,'failed',r.status_code)
    output = ['']*7

  General_ytube_links.loc[i] = list(General_pages.loc[i]) + output

  if i%50 == 0:
     General_ytube_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)

General_ytube_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)




In [ ]:
General_ytube_links.to_csv('/content/gdrive/MyDrive/Amen_Remastered/Song_info.txt', index = False)

In [ ]:
General_ytube_links

,Original_artist,Link,Page,Genre,Sample_type,outcome,expected_vals,actual_vals,New_artist,ytube_link_new,timestamp_new,ytube_link_old,timestamp_old,success,Amen_relation
0,/Drake/,https://www.whosampled.com/Drake/Pound-CakePar...,https://www.whosampled.com/sample/402144/Lil-D...,Multiple Elements,Hip-Hop / Rap / R&B,expected,3,3,/Lil-Dicky/,AOgBg2up7Jg,0:02,VAwWdK7b2zE,0:35,success,False
1,/Drake/,https://www.whosampled.com/Drake/Pound-CakePar...,https://www.whosampled.com/sample/228224/Lupe-...,Multiple Elements,Hip-Hop / Rap / R&B,expected,3,3,/Lupe-Fiasco/,vTWLU1Efsxw,0:00,VAwWdK7b2zE,0:00,success,False
2,/Drake/,https://www.whosampled.com/Drake/Pound-CakePar...,https://www.whosampled.com/sample/238851/Pro-E...,Multiple Elements,Hip-Hop / Rap / R&B,expected,3,3,/Pro-Era/,ndKrLmJheW0,0:51,VAwWdK7b2zE,0:39,success,False
3,/Dr.-Dre/,https://www.whosampled.com/Dr.-Dre/The-Next-Ep...,https://www.whosampled.com/sample/61961/Kanye-...,Vocals / Lyrics,Hip-Hop / Rap / R&B,expected,3,3,/Kanye-West/,wUBhWy0qp5Q,4:42,QZXc39hT8t4,2:06,success,False
4,/Dr.-Dre/,https://www.whosampled.com/Dr.-Dre/The-Next-Ep...,https://www.whosampled.com/sample/3454/City-Hi...,Multiple Elements,Hip-Hop / Rap / R&B,expected,3,3,/City-High/,Gyg5IoEv8oA,1:51,QZXc39hT8t4,2:16,success,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3034,/Timbaland/,https://www.whosampled.com/Timbaland/The-Way-I...,https://www.whosampled.com/sample/26788/Master...,Multiple Elements,Rock / Pop,expected,3,3,/Master-Tempo/,wr0HBt1Obk0,0:27,U5rLz5AZBIA,0:00,success,False
3035,/Timbaland/,https://www.whosampled.com/Timbaland/The-Way-I...,https://www.whosampled.com/sample/89569/Mind-O...,Multiple Elements,Hip-Hop / Rap / R&B,expected,3,3,/Mind-Over-Matter-(Aus)/,,,U5rLz5AZBIA,0:01,partial_2,False
3036,/Aaron-Smith/,https://www.whosampled.com/Aaron-Smith/Dancin/,https://www.whosampled.com/sample/476623/Don-D...,Multiple Elements,Electronic / Dance,expected,3,3,/Don-Diablo/,Oy9V_4im0wg,0:06,N8javR6HO8k,1:28,success,False
3037,/Aaron-Smith/,https://www.whosampled.com/Aaron-Smith/Dancin/,https://www.whosampled.com/sample/992941/Toonl...,Multiple Elements,Electronic / Dance,expected,3,3,/Toonlink/,,,N8javR6HO8k,0:58,partial_2,False


In [ ]:
df = pd.read_csv('Complete_df.txt')
df = df.iloc[:,1:]
df

,Page,Genre,Sample_type,filename,Data_source,Label,Directory
0,https://www.whosampled.com/sample/1142438/Sick...,Electronic / Dance,Drums,1175--Sickboy-Droon-The-Surprise-Remedy,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
1,https://www.whosampled.com/sample/59483/Party-...,Electronic / Dance,Drums,291--Party-Animals-Aquarius,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
2,https://www.whosampled.com/sample/448789/SKisM...,Electronic / Dance,Drums,985--SKisM-Rave-Review-(Ak9-Remix),Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
3,https://www.whosampled.com/sample/87967/Primal...,Rock / Pop,Drums,35--Primal-Scream-Slip-Inside-This-House,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
4,https://www.whosampled.com/sample/551952/Hellf...,Electronic / Dance,Drums,237--Hellfish-My-Style,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
...,...,...,...,...,...,...,...
3624,https://www.whosampled.com/sample/22748/Tuff-C...,Hip-Hop / Rap / R&B,Multiple Elements,755--Tuff-Crew-Tuff-Is-in-the-House-The-Honey-...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
3625,https://www.whosampled.com/sample/265332/Ruki-...,Electronic / Dance,Multiple Elements,590--Ruki-Vverh-(%D0%A0%D1%83%D0%BA%D0%B8-%D0%...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
3626,https://www.whosampled.com/sample/826513/Kawai...,Soundtrack / Library,Vocals / Lyrics,370--Kawai-Sprite-South-Lil-Jon-The-East-Side-...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
3627,https://www.whosampled.com/sample/2670/Heavy-D...,Hip-Hop / Rap / R&B,Hook / Riff,309--Heavy-D-%26-the-Boyz-The-Overweight-Lover...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800


In [ ]:
df.dtypes

Page           object
Genre          object
Sample_type    object
filename       object
Data_source    object
Label           int64
Directory      object
dtype: object

In [ ]:
#arr = [1,2,3]

def plus(x):
  return x + 1

a = plus(5)
print(a)

6


In [ ]:
a = np.array([[1,4,4],[23,55,5], [9,8,6]])
print(a)

df[df['Genre'] == 'Electronic / Dance']







[[ 1  4  4]
 [23 55  5]
 [ 9  8  6]]


,Page,Genre,Sample_type,filename,Data_source,Label,Directory
0,https://www.whosampled.com/sample/1142438/Sick...,Electronic / Dance,Drums,1175--Sickboy-Droon-The-Surprise-Remedy,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
1,https://www.whosampled.com/sample/59483/Party-...,Electronic / Dance,Drums,291--Party-Animals-Aquarius,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
2,https://www.whosampled.com/sample/448789/SKisM...,Electronic / Dance,Drums,985--SKisM-Rave-Review-(Ak9-Remix),Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
4,https://www.whosampled.com/sample/551952/Hellf...,Electronic / Dance,Drums,237--Hellfish-My-Style,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
6,https://www.whosampled.com/sample/59365/Rebel-...,Electronic / Dance,Drums,313--Rebel-MC-Comin%27-on-Strong,Amen,1,Projects\Capstone\Song_files\Songs_with_Amen_1000
...,...,...,...,...,...,...,...
3620,https://www.whosampled.com/sample/88965/Dica-V...,Electronic / Dance,Vocals / Lyrics,227--Dica-Vinyl-Groove-Rush-on-You-The-Jets-Cr...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
3621,https://www.whosampled.com/sample/571366/DJ-Da...,Electronic / Dance,Vocals / Lyrics,149--DJ-Dano-Fukem-All-(Driveby-Mix)-Roxanne-S...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
3622,https://www.whosampled.com/sample/8729/DJ-Mark...,Electronic / Dance,Multiple Elements,164--DJ-Marky-XRS-Highlights-Gilberto-Gil-Realce,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
3623,https://www.whosampled.com/sample/226032/Sub-F...,Electronic / Dance,Vocals / Lyrics,669--Sub-Focus-Safe-in-Sound-Mary-O%27Hara-Oro...,Similar,0,Projects\Capstone\Song_files\Songs_similar_800
